In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
def preprocess(text):
    return text.lower().strip()
df['review']=df['review'].apply(preprocess)

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'sentiment' is a column in your DataFrame df
plt.figure(figsize=(6,6))
df['sentiment'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Sentiment Classes')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.ylabel('')  # Remove the default 'sentiment' label on the y-axis
plt.show()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [8]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq=tokenizer.texts_to_sequences(X_train)
X_test_seq=tokenizer.texts_to_sequences(X_test)

In [9]:
max_len=100
X_train_pad=pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad=pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [11]:
from tensorflow.keras.models import Sequential
model=Sequential([
    Embedding(5000, 128, input_length=max_len),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

Epoch 1/10
625/625 [==============================] - 75s 111ms/step - loss: 0.3919 - accuracy: 0.8178
Epoch 2/10
625/625 [==============================] - 66s 106ms/step - loss: 0.2809 - accuracy: 0.8840
Epoch 3/10
625/625 [==============================] - 68s 108ms/step - loss: 0.2287 - accuracy: 0.9085
Epoch 4/10
625/625 [==============================] - 66s 105ms/step - loss: 0.1905 - accuracy: 0.9251
Epoch 5/10
625/625 [==============================] - 66s 106ms/step - loss: 0.1585 - accuracy: 0.9392
Epoch 6/10
625/625 [==============================] - 69s 110ms/step - loss: 0.1308 - accuracy: 0.9512
Epoch 7/10
625/625 [==============================] - 73s 117ms/step - loss: 0.1098 - accuracy: 0.9605
Epoch 8/10
625/625 [==============================] - 64s 103ms/step - loss: 0.0878 - accuracy: 0.9698
Epoch 9/10
625/625 [==============================] - 66s 105ms/step - loss: 0.0804 - accuracy: 0.9722
Epoch 10/10
625/625 [==============================] - 65s 104ms/step - l

In [13]:
loss, accuracy= model.evaluate(X_test_pad, y_test)
print(loss, accuracy)

313/313 [==============================] - 12s 34ms/step - loss: 0.5982 - accuracy: 0.8559
0.5981548428535461 0.85589998960495


In [21]:
sample="The movie was good"
sample_seq=tokenizer.texts_to_sequences([sample])
sample_pad=pad_sequences(sample_seq, maxlen=max_len, padding='post')
predict=model.predict(sample_pad)
prediction=(predict>0.5).astype(int)[0][0]
final=encoder.inverse_transform([prediction])[0]
print(final)

1/1 [==============================] - 0s 62ms/step
positive
